# 09-Qwen-1_8B-chat CPU 部署

## 概述

本文介绍了在 Intel 设备上部署 Qwen 1.8B 模型的过程，你需要至少16GB内存的机器来完成这项任务，我们将使用英特尔的大模型推理库 [BigDL](https://github.com/intel-analytics/BigDL) 来实现完整过程。

Bigdl-llm 是一个在英特尔设备上运行 LLM（大语言模型）的加速库，通过 INT4/FP4/INT8/FP8 精度量化和架构针对性优化以实现大模型在 英特尔 CPU、GPU上的低资源占用与高速推理能力（适用于任何 PyTorch 模型）。

本文演示为了通用性，只涉及 CPU 相关的代码，如果你想学习如何在 Intel GPU 上部署大模型，可以参考[官网文档](https://bigdl.readthedocs.io/en/latest/doc/LLM/Overview/install_gpu.html)。

## 环境配置

在开始之前，我们需要准备好 bigdl-llm 以及之后部署的相关运行环境，我们推荐你在 python 3.9 的环境中进行之后的操作。

如果你发现下载速度过慢，可以尝试更换默认镜像源：`pip config set global.index-url https://pypi.doubanio.com/simple`

In [8]:
%pip install --pre --upgrade bigdl-llm[all] 
%pip install gradio 
%pip install hf-transfer
%pip install transformers_stream_generator einops
%pip install tiktoken

Looking in indexes: https://pypi.doubanio.com/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.doubanio.com/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.doubanio.com/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.doubanio.com/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.doubanio.com/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


## 模型下载

首先，我们通过 huggingface-cli 获取 qwen-1.8B 模型，耗时较长，我们推荐你先去`喝杯奶茶`。

In [6]:
import os

# 设置环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# 下载模型
os.system('huggingface-cli download --resume-download qwen/Qwen-1_8B-Chat --local-dir qwen18chat_src')

/home/sanbu/test/selfllm/qwen18chat_src


0

## 保存量化模型

为了低资源占用


In [9]:
from bigdl.llm.transformers import AutoModelForCausalLM
from transformers import  AutoTokenizer
import os
if __name__ == '__main__':
    model_path = os.path.join(os.getcwd(),"qwen18chat_src")
    model = AutoModelForCausalLM.from_pretrained(model_path, load_in_low_bit='sym_int4', trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model.save_low_bit('qwen18chat_int4')
    tokenizer.save_pretrained('qwen18chat_int4')

2023-12-18 20:07:19,289 - WARNING - The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
2023-12-18 20:07:19,290 - WARNING - Try importing flash-attention for faster inference...
2023-12-18 20:07:19,290 - WARNING - Warning: import flash_attn rotary fail, please install FlashAttention rotary to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/rotary
2023-12-18 20:07:19,291 - WARNING - Warning: import flash_attn rms_norm fail, please install FlashAttention layer_norm to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/layer_norm
2023-12-18 20:07:19,291 - WARNING - Warning: import flash_attn fail, please install FlashAttention to get higher efficiency https://github.com/Dao-AILab/flash-attention


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2023-12-18 20:07:19,616 - INFO - Converting the current model to sym_int4 format......


## 加载量化模型

In [11]:
import torch
import time
from bigdl.llm.transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

QWEN_PROMPT_FORMAT = "<human>{prompt} <bot>"
load_path = "qwen18chat_int4"
model = AutoModelForCausalLM.load_low_bit(load_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(load_path, trust_remote_code=True)

# 这里可以修改 max_nwe_token
#pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, max_new_tokens=32)
input_str = "给我讲一个年轻人奋斗创业最终取得成功的故事"
with torch.inference_mode():
    prompt = QWEN_PROMPT_FORMAT.format(prompt=input_str)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    st = time.time()
    output = model.generate(input_ids,
                            max_new_tokens=512)
    end = time.time()
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'Inference time: {end-st} s')
    print('-'*20, 'Prompt', '-'*20)
    print(prompt)
    print('-'*20, 'Output', '-'*20)
    print(output_str)

2023-12-18 20:09:35,877 - WARNING - Warning: import flash_attn rotary fail, please install FlashAttention rotary to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/rotary
2023-12-18 20:09:35,878 - WARNING - Warning: import flash_attn rms_norm fail, please install FlashAttention layer_norm to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/layer_norm
2023-12-18 20:09:35,878 - WARNING - Warning: import flash_attn fail, please install FlashAttention to get higher efficiency https://github.com/Dao-AILab/flash-attention


2023-12-18 20:09:36,017 - INFO - Converting the current model to sym_int4 format......


Inference time: 10.887651681900024 s
-------------------- Prompt --------------------
<human>给我讲一个年轻人奋斗创业最终取得成功的故事 <bot>
-------------------- Output --------------------
<human>给我讲一个年轻人奋斗创业最终取得成功的故事 <bot>好的,下面为您讲述一个年轻人奋斗创业最终取得成功的例子——丁磊。

丁磊，1972年出生于浙江杭州，阿里巴巴集团的创始人之一。丁磊的经历充满了挑战和困难，但他从未放弃他的梦想。

他出生在一个普通家庭，父亲是一名工人，母亲是一名教师。然而，他对科技的热情和对成功的渴望驱使他离开家乡去求学。他在大学期间就表现出色，并最终获得了计算机科学硕士学位。在毕业后，他选择了创业道路，创立了自己的互联网公司——网易。

丁磊的事业并非一帆风顺。他在初创时期面临了许多困难和挫折，包括资金短缺、市场竞争激烈等。但是，他坚持自己的信念，始终坚信只有通过不断的努力和坚持，才能实现自己的目标。

在他的努力下，网易公司的产品线逐渐丰富，市场份额也在不断提升。在丁磊的带领下，网易已经成为中国最大的互联网企业之一，也是全球领先的电子商务网站之一。他的成功故事告诉我们，只要我们有坚韧不拔的精神，有勇往直前的决心，就一定能够实现我们的梦想，获得成功。




## gradio-demo 体验


In [13]:
import gradio as gr
import time
from bigdl.llm.transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

QWEN_PROMPT_FORMAT = "<human>{prompt} <bot>"

load_path = "qwen18chat_int4"
model = AutoModelForCausalLM.load_low_bit(load_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(load_path,trust_remote_code=True)

def add_text(history, text):
    _, history = model.chat(tokenizer, text, history=history)
    return history, gr.Textbox(value="", interactive=False)

def bot(history):
    response =  history[-1][1]
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [], 
        elem_id="chatbot",
        bubble_full_width=False,
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="Enter text and press enter",
            container=False,
        )

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot, api_name="bot_response"
    )
    txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)

demo.queue()
demo.launch()

2023-12-18 20:12:34,418 - WARNING - Warning: import flash_attn rotary fail, please install FlashAttention rotary to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/rotary
2023-12-18 20:12:34,418 - WARNING - Warning: import flash_attn rms_norm fail, please install FlashAttention layer_norm to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/layer_norm
2023-12-18 20:12:34,419 - WARNING - Warning: import flash_attn fail, please install FlashAttention to get higher efficiency https://github.com/Dao-AILab/flash-attention
2023-12-18 20:12:34,558 - INFO - Converting the current model to sym_int4 format......
2023-12-18 20:12:35,857 - INFO - HTTP Request: GET http://127.0.0.1:7861/startup-events "HTTP/1.1 200 OK"
2023-12-18 20:12:35,862 - INFO - HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


2023-12-18 20:12:36,544 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2023-12-18 20:12:36,680 - INFO - HTTP Request: POST https://api.gradio.app/gradio-initiated-analytics/ "HTTP/1.1 200 OK"
2023-12-18 20:12:36,796 - INFO - HTTP Request: POST https://api.gradio.app/gradio-launched-telemetry/ "HTTP/1.1 200 OK"
